1) You previously created an inverted index of terms and pages; this time, create a dictionary that contains an inverted index from the values contained in web_graph_small.txt. Assume the number in the first column is linking to the second. 

You can set either the first number as the key and all nodes it links to as a list of values, or the other way around -- just be sure you note explicitly which it is in comments to keep track.  In fact, some people find it easier for calculation to make two inverted indexes (one for nodes:outlinks, one for nodes:inlinks, but you do you).

In [1]:
from nltk import flatten

In [2]:
import sys
from random import randrange
sys.setrecursionlimit(randrange(2500, 3000))

In [3]:
with open ('small.txt','r') as file:
    content = [line.strip().split('\t') for line in file.readlines()]

In [4]:
out_degree = {}
for (key, value) in content:
    out_degree.setdefault(key, []).append(value)
out_degree

{'1': ['3', '2'], '2': ['3'], '4': ['3'], '3': ['1']}

In [5]:
in_degree = {}
for (key, value) in content:
    in_degree.setdefault(value, []).append(key)
in_degree

{'3': ['1', '2', '4'], '2': ['1'], '1': ['3']}


2) Using this (or these) inverted index/indices of links, create a function that calculates a dictionary, with node numbers as keys and rank weights as values:

a) initially, all nodes should have the same weight: 1/n, where n is the number of nodes.

In [12]:
rank = {}

In [13]:
def pagerank(out_degree, in_degree):
    node = set(flatten(content))
    n = len(node)
    for each in node:
        rank[each] = 1/n
    #rank = dict.fromkeys(node, 1/n) 
    return rank

In [14]:
rank = pagerank(out_degree, in_degree)
rank

{'2': 0.25, '3': 0.25, '1': 0.25, '4': 0.25}

b) Code one cycle of rank value updates, using a value for p (the probability that a user will go directly to a node rather than following a link to it) of 0.2.

c) Once you have one cycle of rank updates working, alter your function so that it calls itself, feeding the output of one cycle into the next cycle as input, until the rank vector of a cycle is the same as the ranks from the previous cycle. Build in a trace so you can track the number of cycles. How many did it take?

In [15]:
counter = 0

In [16]:
def pagerank(out_degree, in_degree, rank, p):
    global counter
    node = set(flatten(content))
    n = len(node)
    
    check = rank.copy()
    counter += 1
    
    out_amount = {}
    for key, value in out_degree.items():
        out_amount[key] = len(value)
       
    in_amount = {}
    for k, value in in_degree.items():
        for each in value:
            in_amount.setdefault(k, []).append(float(str(rank.get(each)))/int(out_amount.get(each)))
    in_amount = {k: sum(map(float, (i for i in v))) for k, v in in_amount.items()}
    for each in rank.keys():
        if each in in_amount.keys():
            rank[each] = p/n + (1-p) * (in_amount[each])
        else:
            rank[each] = p/n + (1-p) * 0
    for key in rank.keys():
        rank[key] = round(rank[key],2)      
    
    if check != rank:
        print('rank is',rank)
        print('check is',check)
        print('number of cycle:', counter)
        rank = pagerank(out_degree, in_degree, rank, p)
    else:
        return rank, counter
    

In [11]:
rank = pagerank(out_degree, in_degree, rank, .2)
rank

rank is {'2': 0.15, '3': 0.55, '1': 0.25, '4': 0.05}
check is {'2': 0.25, '3': 0.25, '1': 0.25, '4': 0.25}
number of cycle: 1
rank is {'2': 0.15, '3': 0.31, '1': 0.49, '4': 0.05}
check is {'2': 0.15, '3': 0.55, '1': 0.25, '4': 0.05}
number of cycle: 2
rank is {'2': 0.25, '3': 0.41, '1': 0.3, '4': 0.05}
check is {'2': 0.15, '3': 0.31, '1': 0.49, '4': 0.05}
number of cycle: 3
rank is {'2': 0.17, '3': 0.41, '1': 0.38, '4': 0.05}
check is {'2': 0.25, '3': 0.41, '1': 0.3, '4': 0.05}
number of cycle: 4
rank is {'2': 0.2, '3': 0.38, '1': 0.38, '4': 0.05}
check is {'2': 0.17, '3': 0.41, '1': 0.38, '4': 0.05}
number of cycle: 5
rank is {'2': 0.2, '3': 0.4, '1': 0.35, '4': 0.05}
check is {'2': 0.2, '3': 0.38, '1': 0.38, '4': 0.05}
number of cycle: 6
rank is {'2': 0.19, '3': 0.39, '1': 0.37, '4': 0.05}
check is {'2': 0.2, '3': 0.4, '1': 0.35, '4': 0.05}
number of cycle: 7
rank is {'2': 0.2, '3': 0.39, '1': 0.36, '4': 0.05}
check is {'2': 0.19, '3': 0.39, '1': 0.37, '4': 0.05}
number of cycle: 8
r



d) Adjust the value of p to 0.05, and run again. Does the number of cycles change?

In [17]:
second_rank = pagerank(out_degree, in_degree, rank, .05)
second_rank

rank is {'2': 0.13, '3': 0.61, '1': 0.25, '4': 0.01}
check is {'2': 0.25, '3': 0.25, '1': 0.25, '4': 0.25}
number of cycle: 1
rank is {'2': 0.13, '3': 0.26, '1': 0.59, '4': 0.01}
check is {'2': 0.13, '3': 0.61, '1': 0.25, '4': 0.01}
number of cycle: 2
rank is {'2': 0.29, '3': 0.43, '1': 0.26, '4': 0.01}
check is {'2': 0.13, '3': 0.26, '1': 0.59, '4': 0.01}
number of cycle: 3
rank is {'2': 0.14, '3': 0.42, '1': 0.42, '4': 0.01}
check is {'2': 0.29, '3': 0.43, '1': 0.26, '4': 0.01}
number of cycle: 4
rank is {'2': 0.21, '3': 0.35, '1': 0.41, '4': 0.01}
check is {'2': 0.14, '3': 0.42, '1': 0.42, '4': 0.01}
number of cycle: 5
rank is {'2': 0.21, '3': 0.42, '1': 0.34, '4': 0.01}
check is {'2': 0.21, '3': 0.35, '1': 0.41, '4': 0.01}
number of cycle: 6
rank is {'2': 0.17, '3': 0.38, '1': 0.41, '4': 0.01}
check is {'2': 0.21, '3': 0.42, '1': 0.34, '4': 0.01}
number of cycle: 7
rank is {'2': 0.21, '3': 0.38, '1': 0.37, '4': 0.01}
check is {'2': 0.17, '3': 0.38, '1': 0.41, '4': 0.01}
number of c